In [686]:
import requests
import json
from dotenv import load_dotenv
import os
import pandas as pd

In [687]:
from geopy.distance import distance

#### Cargamos el DataFrame de successful startups filtrado con los criterios de:
    - Empresas fundadas después de 2010
    - Empresas que hayan logrado $1M de financiación

In [457]:
df = pd.read_csv("Data/successful_startups_dataset.csv")

In [418]:
df.longitudes[0]

-73.985506

#### Primero debemos registrarnos en FourSquare y cargamos nuestros tokens al .env. 


In [420]:
load_dotenv()

True

In [421]:
tok1 = os.getenv("tok1")
tok2 = os.getenv("tok2")

#### Probamos a hacer una llamada a la API

In [422]:
url = "https://api.foursquare.com/v2/venues/search"

In [423]:
parametros = {"client_id" : tok1,
              "client_secret" : tok2,
              "v" : "20180323",
              "ll" : f"{df.latitudes[0]},{df.longitudes[0]}",
              "radius" : 5000,
              "categoryId" : "4bf58dd8d48988d1e0931735",
              "limit" : 100
             }

In [503]:
resp = requests.get (url=url, params = parametros)
data = json.loads(resp.text)
data

{'meta': {'code': 200, 'requestId': '5fb2acaaa5421e17ccd48939'},
 'response': {'venues': [{'id': '3fd66200f964a52035e91ee3',
    'name': 'China Club',
    'contact': {},
    'location': {'address': '268 W 47th St',
     'lat': 40.760169982910156,
     'lng': -73.98715209960938,
     'labeledLatLngs': [{'label': 'display',
       'lat': 40.760169982910156,
       'lng': -73.98715209960938}],
     'distance': 285,
     'postalCode': '10036',
     'cc': 'US',
     'city': 'New York',
     'state': 'NY',
     'country': 'United States',
     'formattedAddress': ['268 W 47th St',
      'New York, NY 10036',
      'United States']},
    'categories': [{'id': '4bf58dd8d48988d11f941735',
      'name': 'Nightclub',
      'pluralName': 'Nightclubs',
      'shortName': 'Nightclub',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/nightlife/nightclub_',
       'suffix': '.png'},
      'primary': True}],
    'verified': False,
    'stats': {'tipCount': 0,
     'usersCount': 0,
     

#### Vemos como vamos a limpiar esa información:

In [504]:
data.keys()

dict_keys(['meta', 'response'])

In [505]:
decoding_data = data.get("response")
#Ahora ya tenemos un diccionario solo en lugar de dos.
#Ahora queremos bucear a coger solo lo que nos interesa:
decoding_data2 = decoding_data.get("venues")
#Seguimos buceando. Ahora tenemos dentro de venues una lista de diccionarios
decoding_data2[2].get("name")

'Tonic'

### Ya solo tenemos que contruir funciones que extraigan la información que queremos comprobar para todas las ubicaciones presentes en el DataFrame filtrado. 

### Los criterios son los siguientes:
    - Un starbucks a menos de 500 metros 
    - Un estadio de baloncesto a menos de 10 kilómetros
    - Un aeropuerto a menos de 5 kilómetros
    - Una guardería a menos de 500 metros
    - Discotecas a menos de 1 kilómetros
    - Un restaurante vegano a menos de 500 metros

#### Un starbucks a menos de 500 metros

In [462]:
def is_there_starbucks(list, dataframe):        
        for i in range(0, (len(dataframe))):    
            parametros = {"client_id" : tok1,
                          "client_secret" : tok2,
                          "v" : "20180323",
                          "ll" : f"{dataframe.latitudes[i]},{dataframe.longitudes[i]}",
                          "radius" : 500,
                          "categoryId" : "4bf58dd8d48988d1e0931735",
                          "limit" : 100
                         }
            resp = requests.get (url=url, params = parametros)
            data = json.loads(resp.text)
            decoding_data = data.get("response")
            decoding_data2 = decoding_data.get("venues")
            counter = 0
            for i in range(0, len(decoding_data2)):
                if "Starbucks" in (decoding_data2[i].get("name")):
                    counter += 1
            if counter > 0:
                list.append(1)
            elif counter == 0:
                list.append(0)

In [463]:
starbucks_nearby = []
is_there_starbucks(starbucks_nearby, df)
starbucks_nearby

[1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0]

In [497]:
#Checkeamos que estamos sacando 11 respuestas, al ser nuestro df de 11 localizaciones. 
len(starbucks_nearby) == len(df)

True

#### Un estadio de baloncesto a menos de 10 kilómetros

In [487]:
def is_there_basketball_stadium(list, dataframe):
        for i in range(0, (len(dataframe))):    
            parametros = {"client_id" : tok1,
                          "client_secret" : tok2,
                          "v" : "20180323",
                          "ll" : f"{dataframe.latitudes[i]},{dataframe.longitudes[i]}",
                          "radius" : 10000,
                          "categoryId" : "4bf58dd8d48988d18b941735",
                          "limit" : 100
                         }
            resp = requests.get (url=url, params = parametros)
            data = json.loads(resp.text)
            decoding_data = data.get("response")
            decoding_data2 = decoding_data.get("venues")
            counter = 0
            if len(decoding_data2) > 0:
                list.append(1)
            else:
                list.append(0)

In [488]:
basketball_stadium_nearby = []
is_there_basketball_stadium(basketball_stadium_nearby, df)
basketball_stadium_nearby

[1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1]

#### Un aeropuerto a menos de 5 kilómetros

In [485]:
def is_there_airport(list, dataframe):
        for i in range(0, (len(dataframe))):    
            parametros = {"client_id" : tok1,
                          "client_secret" : tok2,
                          "v" : "20180323",
                          "ll" : f"{dataframe.latitudes[i]},{dataframe.longitudes[i]}",
                          "radius" : 5000,
                          "categoryId" : "4bf58dd8d48988d1ed931735",
                          "limit" : 100
                         }
            resp = requests.get (url=url, params = parametros)
            data = json.loads(resp.text)
            decoding_data = data.get("response")
            decoding_data2 = decoding_data.get("venues")
            counter = 0
            if len(decoding_data2) > 0:
                list.append(1)
            else:
                list.append(0)

In [486]:
airport_nearby = []
is_there_airport(airport_nearby, df)
airport_nearby

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

#### Una guardería a menos de 500 metros

In [498]:
def is_there_child_care(list, dataframe):
        for i in range(0, (len(dataframe))):    
            parametros = {"client_id" : tok1,
                          "client_secret" : tok2,
                          "v" : "20180323",
                          "ll" : f"{dataframe.latitudes[i]},{dataframe.longitudes[i]}",
                          "radius" : 500,
                          "categoryId" : "5744ccdfe4b0c0459246b4c7",
                          "limit" : 100
                         }
            resp = requests.get (url=url, params = parametros)
            data = json.loads(resp.text)
            decoding_data = data.get("response")
            decoding_data2 = decoding_data.get("venues")
            if len(decoding_data2) > 0:
                list.append(1)
            else:
                list.append(0)

In [499]:
child_care_nearby = []
is_there_child_care(child_care_nearby, df)
child_care_nearby

[1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0]

#### Discotecas a menos de 1 kilómetros

In [500]:
def is_there_nightclubs(list, dataframe):
        for i in range(0, (len(dataframe))):    
            parametros = {"client_id" : tok1,
                          "client_secret" : tok2,
                          "v" : "20180323",
                          "ll" : f"{dataframe.latitudes[i]},{dataframe.longitudes[i]}",
                          "radius" : 1000,
                          "categoryId" : "4bf58dd8d48988d11f941735",
                          "limit" : 100
                         }
            resp = requests.get (url=url, params = parametros)
            data = json.loads(resp.text)
            decoding_data = data.get("response")
            decoding_data2 = decoding_data.get("venues")
            if len(decoding_data2) > 0:
                list.append(1)
            else:
                list.append(0)

In [501]:
nightclubs_nearby = []
is_there_nightclubs(nightclubs_nearby, df)
nightclubs_nearby

[1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0]

#### Un restaurante vegano a menos de 500 metros

In [506]:
def is_there_vegan_rest(list, dataframe):
        for i in range(0, (len(dataframe))):    
            parametros = {"client_id" : tok1,
                          "client_secret" : tok2,
                          "v" : "20180323",
                          "ll" : f"{dataframe.latitudes[i]},{dataframe.longitudes[i]}",
                          "radius" : 500,
                          "categoryId" : "4bf58dd8d48988d1d3941735",
                          "limit" : 100
                         }
            resp = requests.get (url=url, params = parametros)
            data = json.loads(resp.text)
            decoding_data = data.get("response")
            decoding_data2 = decoding_data.get("venues")
            if len(decoding_data2) > 0:
                list.append(1)
            else:
                list.append(0)

In [507]:
vegan_rest_nearby = []
is_there_vegan_rest(vegan_rest_nearby, df)
vegan_rest_nearby

[1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0]

### Ahora que tenemos listas con el cumplimiento de los diferentes criterios observados los introducimos a nuestro DataFrame actualizado.

In [508]:
df["Starbucks?"] = starbucks_nearby
df["Basketball Stadium?"] = basketball_stadium_nearby
df["Airport?"] = airport_nearby
df["Child Care?"] = child_care_nearby
df["Nightclubs?"] = nightclubs_nearby
df["Vegan Restaurants"] = vegan_rest_nearby

In [530]:
#Aqui me quito una columna que me ha salido de más
df.drop(["Unnamed: 0"], axis = 1);

#### Ahora debemos crear una nueva columna que establezca la puntuación de cada localización 

In [515]:
df["Overall Score"] = df["Starbucks?"] + df["Basketball Stadium?"] + df["Airport?"] + df["Child Care?"] + df["Nightclubs?"] + df["Vegan Restaurants"]

#### Y con pandas ordenamos los resultados en base a esta puntuación, comenzando con el de máxima puntuación.

In [523]:
sort_by_overall_score = df.sort_values("Overall Score", ascending = False)

In [524]:
sort_by_overall_score

,Unnamed: 0,name,category_code,founded_year,total_money_raised,offices,coordinates,latitudes,longitudes,Starbucks?,Basketball Stadium?,Airport?,Child Care?,Nightclubs?,Vegan Restaurants,Overall Score
0,0,PeekYou,search,2012,$1.83M,"[{'latitude': 40.757929, 'longitude': -73.9855...","[40.757929, -73.985506]",40.757929,-73.985506,1,1,1,1,1,1,6
4,4,Social Gaming Network,games_video,2011,$17.1M,"[{'latitude': 37.446823, 'longitude': -122.161...","[37.446823, -122.161523]",37.446823,-122.161523,1,1,1,1,1,1,6
2,2,Widgetbox,other,2012,$14.5M,"[{'latitude': 37.798853, 'longitude': -122.398...","[37.798853, -122.398599]",37.798853,-122.398599,1,1,1,0,1,1,5
3,3,RazorGator,ecommerce,2011,$58.8M,"[{'latitude': 34.047312, 'longitude': -118.445...","[34.047312, -118.445243]",34.047312,-118.445243,1,1,1,0,1,1,5
1,1,Fixya,web,2013,$8M,"[{'latitude': 37.566879, 'longitude': -122.323...","[37.566879, -122.323895]",37.566879,-122.323895,1,0,1,1,1,0,4
5,5,Skydeck,mobile,2012,$4M,"[{'latitude': 37.5645375, 'longitude': -122.32...","[37.5645375, -122.3254705]",37.564538,-122.325471,1,0,1,1,1,0,4
6,6,Moblica,mobile,2011,$6M,"[{'latitude': 32.0554, 'longitude': 34.7595}]","[32.0554, 34.7595]",32.055400,34.759500,0,1,1,0,1,1,4
7,7,travelmob,travel,2011,$1M,"[{'latitude': 21.303049, 'longitude': -157.789...","[21.303049, -157.78907]",21.303049,-157.789070,0,1,1,0,0,0,2
8,8,travelmob,travel,2011,$1M,"[{'latitude': 21.303049, 'longitude': -157.789...","[21.303049, -157.78907]",21.303049,-157.789070,0,1,1,0,0,0,2
9,9,VisualOn,software,2011,$3.1M,"[{'latitude': 37.270518, 'longitude': -121.955...","[37.270518, -121.955879]",37.270518,-121.955879,0,0,1,0,1,0,2


In [529]:
#Ponemos la puntuación de índice
final_scores = sort_by_overall_score.set_index("Overall Score")
#Nos quitamos el índice antiguo
final_scores.drop("Unnamed: 0", axis =1)

,name,category_code,founded_year,total_money_raised,offices,coordinates,latitudes,longitudes,Starbucks?,Basketball Stadium?,Airport?,Child Care?,Nightclubs?,Vegan Restaurants
Overall Score,,,,,,,,,,,,,,
6,PeekYou,search,2012,$1.83M,"[{'latitude': 40.757929, 'longitude': -73.9855...","[40.757929, -73.985506]",40.757929,-73.985506,1,1,1,1,1,1
6,Social Gaming Network,games_video,2011,$17.1M,"[{'latitude': 37.446823, 'longitude': -122.161...","[37.446823, -122.161523]",37.446823,-122.161523,1,1,1,1,1,1
5,Widgetbox,other,2012,$14.5M,"[{'latitude': 37.798853, 'longitude': -122.398...","[37.798853, -122.398599]",37.798853,-122.398599,1,1,1,0,1,1
5,RazorGator,ecommerce,2011,$58.8M,"[{'latitude': 34.047312, 'longitude': -118.445...","[34.047312, -118.445243]",34.047312,-118.445243,1,1,1,0,1,1
4,Fixya,web,2013,$8M,"[{'latitude': 37.566879, 'longitude': -122.323...","[37.566879, -122.323895]",37.566879,-122.323895,1,0,1,1,1,0
4,Skydeck,mobile,2012,$4M,"[{'latitude': 37.5645375, 'longitude': -122.32...","[37.5645375, -122.3254705]",37.564538,-122.325471,1,0,1,1,1,0
4,Moblica,mobile,2011,$6M,"[{'latitude': 32.0554, 'longitude': 34.7595}]","[32.0554, 34.7595]",32.055400,34.759500,0,1,1,0,1,1
2,travelmob,travel,2011,$1M,"[{'latitude': 21.303049, 'longitude': -157.789...","[21.303049, -157.78907]",21.303049,-157.789070,0,1,1,0,0,0
2,travelmob,travel,2011,$1M,"[{'latitude': 21.303049, 'longitude': -157.789...","[21.303049, -157.78907]",21.303049,-157.789070,0,1,1,0,0,0


#### Vemos que tenemos un empate entre dos localizaciones con la misma puntuación, ya que ambas cumplen todos los criterios

In [745]:
#Vamos a guardar las localizaciones de los dos lugares ganadores
lat1 = 40.757929
long1 = -73.985506
lat2 = 37.446823
long2 = -122.161523

#### Intentemos ser más específicos:
    - Distancia exacta de ambos al Starbucks más cercano

Definimos una función para calcular la distancia al Starbucks más cercano

In [753]:
def distance_closest_starbucks(lat, long):
    parametros = {"client_id" : tok1,
                          "client_secret" : tok2,
                          "v" : "20180323",
                          "ll" : f"{lat},{long}",
                          "radius" : 5000,
                          "categoryId" : "4bf58dd8d48988d1e0931735",
                          "limit" : 100
                         }
    resp = requests.get (url=url, params = parametros)
    data = json.loads(resp.text)
    decoding_data = data.get("response")
    decoding_data2 = decoding_data.get("venues")
    distances = []
    for i in range(0, len(decoding_data2)):
        if "Starbucks" in decoding_data2[i].get("name"):
            location_starbucks1 = decoding_data2[i].get("location")
            lat_starbucks1 = location_starbucks1.get("lat")
            lng_starbucks1 = location_starbucks1.get("lng")
            coordinates1 = (lat, long)
            coordinates_starbucks1 = (lat_starbucks1, lng_starbucks1)
            distances.append(distance(coordinates1, coordinates_starbucks1).km)
    return min(distances)

In [750]:
distance_closest_starbucks(lat1, long1)

0.0479739117306134

In [751]:
distance_closest_starbucks(lat2, long2)

0.09195805978002843

### Decisión final: (40.757929, -73.985506)

In [ ]:
#A PARTIR DE AQUÍ SUCIO

In [755]:
    parametros = {"client_id" : tok1,
                          "client_secret" : tok2,
                          "v" : "20180323",
                          "ll" : f"{lat1},{long1}",
                          "radius" : 1000,
                          "categoryId" : "4bf58dd8d48988d1e0931735",
                          "limit" : 1000
                         }
    resp = requests.get (url=url, params = parametros)
    data = json.loads(resp.text)
    decoding_data = data.get("response")
    decoding_data2 = decoding_data.get("venues")

In [756]:
decoding_data2

[{'id': '4b5f42a1f964a52094b029e3',
  'name': 'Starbucks',
  'contact': {},
  'location': {'address': '1585 Broadway',
   'crossStreet': 'at W 47th St',
   'lat': 40.760041,
   'lng': -73.985579,
   'labeledLatLngs': [{'label': 'display',
     'lat': 40.760041,
     'lng': -73.985579},
    {'label': 'entrance', 'lat': 40.759823, 'lng': -73.985106}],
   'distance': 235,
   'postalCode': '10036',
   'cc': 'US',
   'city': 'New York',
   'state': 'NY',
   'country': 'United States',
   'formattedAddress': ['1585 Broadway (at W 47th St)',
    'New York, NY 10036',
    'United States']},
  'categories': [{'id': '4bf58dd8d48988d1e0931735',
    'name': 'Coffee Shop',
    'pluralName': 'Coffee Shops',
    'shortName': 'Coffee Shop',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/coffeeshop_',
     'suffix': '.png'},
    'primary': True}],
  'verified': True,
  'stats': {'tipCount': 0,
   'usersCount': 0,
   'checkinsCount': 0,
   'visitsCount': 0},
  'beenHere': {'count': 

In [734]:
distances = []
for i in range(0, len(decoding_data2)):
    if "Starbucks" in decoding_data2[i].get("name"):
        location_starbucks1 = decoding_data2[i].get("location")
        lat_starbucks1 = location_starbucks1.get("lat")
        lng_starbucks1 = location_starbucks1.get("lng")
        coordinates1 = (lat1, long1)
        coordinates_starbucks1 = (lat_starbucks1, lng_starbucks1)
        distances.append(distance(coordinates1, coordinates_starbucks1).km)
print(min(distances))

[0.23461701162245752, 0.29027876992733453, 0.2908972239453193, 1.0907369756743595, 0.48128503139383644, 1.0674457795148595, 0.6023648813170079, 1.2542932657241581, 1.2304260999241627, 1.0686211656686924, 0.42428533336224006, 1.0232833839372995, 0.5350552732290536, 0.9864880765241711, 0.6432993480459932, 0.9442868635140169, 1.023498175172935, 0.7171685240377023, 1.012743400384208, 0.8197038433994921]
0.23461701162245752
